# Module 1 - Thu thập dữ liệu từ [batdongsan.com.vn](https://batdongsan.com.vn/cho-thue-can-ho-chung-cu)

## 1. Mục tiêu

### 1.1. Kịch bản

Thực hiện thu thập dữ liệu chung cư cho thuê ở thành phố Hồ Chí Minh từ [nhatot.vn](https://www.nhatot.com/thue-can-ho-chung-cu-tp-ho-chi-minh), xem xét sơ bộ ta thấy:

+ Mỗi trang sẽ chứa 20 tin rao cho thuê, và có khoảng 1000 trang
+ Thông tin cần phải thu thập cho mỗi căn hộ:

| Columns            | Description               |
|--------------------|---------------------------|
| ad_id              | id của tin (để phân biệt) |
| list_id            | id trong danh sách        |
| subject            | Tiêu đề của tin           |
| price              | Giá thuê                  |
| size               | Diện tích                 |
| rooms              | Số phòng                  |
| toilets            | Số nhà vệ sinh            |
| area_name          | Quận huyện                |
| region_name        | Thành phố, tỉnh           |
| ward_name          | Phường                    |
| street_name        | Đuờng                     |
| detail_address     | Địa chỉ chi tiết          |
| zero_deposit       | Có cần tiền cọc không?    |
| escrow_can_deposit | Cọc bao nhiêu tháng       |
| longitude          | longitude                 |
| latitude           | latitude                  |

## 2. Thực hiện

### 2.1. Import thư viện cần thiết

In [80]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait

import csv
import time
import json

### 2.2. CONSTANT

In [81]:
SLEEP_TIME = 2
BASE_URL='https://www.nhatot.com/thue-can-ho-chung-cu-tp-ho-chi-minh'
LIMIT_PAGE = 1000

### 2.3. Init chrome driver

In [104]:
# Các option cho chrome: ẩn danh, cửa sổ
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"

options = webdriver.ChromeOptions()
# options.headless = True
options.add_argument(f'user-agent={user_agent}')
options.add_argument("--window-size=1920,1080")
options.add_argument('--ignore-certificate-errors')
options.add_argument('--allow-running-insecure-content')
options.add_argument("--disable-extensions")
options.add_argument("--proxy-server='direct://'")
options.add_argument("--proxy-bypass-list=*")
options.add_argument("--start-maximized")

options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')

# khởi tạo driver
driver = webdriver.Chrome(chrome_options=options, executable_path='./driver/chromedriver')

/var/folders/cf/tjy8lz4n39q0hzxh7gb0n9tr0000gn/T/ipykernel_24206/2849229068.py:20: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path='./driver/chromedriver')


### 2.3. Craw data

Cách thực hiện:
    + Do mỗi trang có sẵn một script tag với id '__NEXT_DATA__' chứa thông tin chi tiết của 20 căn hộ trong danh sách, nên ta chỉ cần request tới trang danh sách và lấy dữ liệu mà không cần vào chi trang chi tiết
    + Vì trang web sử dụng CloudFlare security nên với chỉ mở một browser duy nhất, mỗi lần crawl xong dữ liệu cần tìm next button để chuyển qua trang tiếp theo

In [105]:
# get all apartments from 1000 pages
def get_data_per_page(limit):
    all_apartments = []
    page = 1
    # Go to page
    driver.get(BASE_URL)
    time.sleep(SLEEP_TIME)
    # Scroll to end of page (for finding next button)
    driver.execute_script("window.scrollBy(0,document.body.scrollHeight)","")
    time.sleep(SLEEP_TIME)

    while page <= limit:
        print(f'Get data from page {page}')
        # get raw data from a script tag
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        raw_data = page_source.find('script', id='__NEXT_DATA__').string
        # convert raw to dict
        data = json.loads(raw_data)
        # get list apartment
        list_apartment = data['props']['initialState']['adlisting']['data']['ads']
        # add to results
        all_apartments.extend(list_apartment)
        # find next button need to wait page load fully
        back_next_buttons = driver.find_elements_by_class_name("Paging_redirectPageBtn__KvsqJ")
        while len(back_next_buttons) < 2:
            time.sleep(SLEEP_TIME)
            back_next_buttons = driver.find_elements_by_class_name("Paging_redirectPageBtn__KvsqJ")

        next_button = back_next_buttons[1]
        # navigate to next page
        driver.execute_script("arguments[0].click();", next_button)
        page += 1
        time.sleep(SLEEP_TIME)

    driver.close()
    driver.quit()
    return all_apartments

In [106]:
apartments = get_data_per_page(LIMIT_PAGE)
apartments

Get data from page 1
Get data from page 2
Get data from page 3
Get data from page 4
Get data from page 5
Get data from page 6
Get data from page 7
Get data from page 8
Get data from page 9
Get data from page 10
Get data from page 11
Get data from page 12
Get data from page 13
Get data from page 14
Get data from page 15
Get data from page 16
Get data from page 17
Get data from page 18
Get data from page 19
Get data from page 20
Get data from page 21
Get data from page 22
Get data from page 23
Get data from page 24
Get data from page 25
Get data from page 26
Get data from page 27
Get data from page 28
Get data from page 29
Get data from page 30
Get data from page 31
Get data from page 32
Get data from page 33
Get data from page 34
Get data from page 35
Get data from page 36
Get data from page 37
Get data from page 38
Get data from page 39
Get data from page 40
Get data from page 41
Get data from page 42
Get data from page 43
Get data from page 44
Get data from page 45
Get data from page 

[{'ad_id': 140064645,
  'list_id': 101581228,
  'list_time': None,
  'date': '2 phút trước',
  'account_id': 12094115,
  'projectid': 1661,
  'project_oid': '131318744',
  'account_oid': None,
  'account_name': 'Thanh Trúc',
  'subject': 'Căn hộ cho thuê giá 7.5 triệu Khang Gia Gò Vấp',
  'body': None,
  'category': 1010,
  'category_name': None,
  'area': 110,
  'area_name': 'Quận Gò Vấp',
  'region': 13,
  'region_name': 'Tp Hồ Chí Minh',
  'company_ad': True,
  'type': 'u',
  'price': 7500000,
  'price_string': '7,5 triệu/tháng',
  'image': 'https://cdn.chotot.com/RlTA6SsqvLx1jAbFc557n-rLNRa5-u4mpppK6nmx8bM/preset:listing/plain/c0581baa7215295682e71cb3ff93f8f1-2801431311530665892.jpg',
  'webp_image': 'https://cdn.chotot.com/2BwsaqxuliHbpITj2PGMUzN3ocz4UZH7yDcxSqZEuj8/preset:listing/plain/c0581baa7215295682e71cb3ff93f8f1-2801431311530665892.webp',
  'videos': [],
  'number_of_images': 9,
  'avatar': 'https://cdn.chotot.com/uac2/12094115',
  'rooms': 2,
  'apartment_type': 1,
  'size

## 2.4. Filter data

In [107]:
need_keys = ['ad_id', 'list_id', 'subject', 'price', 'size', 'rooms', 'toilets', 'area_name', 'region_name', 'ward_name', 'street_name', 'detail_address', 'escrow_can_deposit', 'zero_deposit', 'longitude', 'latitude']

def filer_apartment(apartment):
    return { key: apartment[key] if key in apartment else '' for key in need_keys }

new_apartments = list(map(filer_apartment, apartments))
new_apartments[0]

{'ad_id': 140064645,
 'list_id': 101581228,
 'subject': 'Căn hộ cho thuê giá 7.5 triệu Khang Gia Gò Vấp',
 'price': 7500000,
 'size': 76,
 'rooms': 2,
 'toilets': 2,
 'area_name': 'Quận Gò Vấp',
 'region_name': 'Tp Hồ Chí Minh',
 'ward_name': 'Phường 14',
 'street_name': 'Số 45',
 'detail_address': 'Đường Số 45, Phường 14, Gò Vấp, Hồ Chí Minh',
 'escrow_can_deposit': 2,
 'zero_deposit': False,
 'longitude': 106.63781,
 'latitude': 10.8431}

### 2.5. Write to csv

In [108]:
list_apartment = [[apartment[key] for key in need_keys] for apartment in new_apartments]

with open('./data/hcm-apartment-rent-data.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(need_keys)
    write.writerows(list_apartment)